In [1]:
import pandas as pd
import numpy as np

In [23]:
db = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 сш.xlsx").map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

In [24]:
to_read = {
    # 26 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жамбылская область\COPY - 4-8_сынып_БЖБ-ТЖБ Ә.Бөкейхан атындағы  №15 тірек мектеп-лицей.xlsx", # Жамбылская область
    27 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жамбылская область\COPY 2 - Мойынкум 4 -8  Мойынқұм ауданы, Т.Рысқұлов тірек.xlsx", # Жамбылская область
    # 30 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жетисуская область\COPY - СОР и СОЧ -шаблон Когалинская СШ2.xlsx", # Жетисуская область
}

In [25]:
# to_read = {42 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Аққулы СОР и СОЧ.xlsx",}
df_dict = {}
ss_list = ['sor1', 'sor2', 'sor3', 'soch']
for key, file_path in to_read.items():
    try:
        df_dict[f"df{key}"] = pd.read_excel(file_path, sheet_name='data', dtype={col: str for col in ss_list})
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file: {file_path}. Error: {e}")

In [26]:
# Leaving only First anf Last names of each student
db['fio'] = db['fio'].map(lambda x: " ".join(str(x).split()[:2]) if len(str(x).split()) > 2 else x)

In [27]:
subject_map = {
    'английский язык' : 'ENFL',
    'ағылшын тілі' : 'ENFL',
    'биология' : 'BIOL',
    'география' : 'GEOG',
    'жаратылыстану' : 'SCIE',
    'казахский язык' : 'KZFL',
    'математика' : 'MATH',
    'русский язык' : 'RUFL',
    'русский язык и литература' : 'RUFL',
    'физика' : 'PHYS',
    'химия' : 'CHEM',
    'қазақ тілі' : 'KZFL'
}

In [29]:
df_dict['df27'] 

,fio,subject,sor1,sor2,sor3,soch
0,Абай Аяна,8CHEM,13--4,10--7,6--4,25-20
1,Абай Аяна,8BIOL,25--19,10--7,15--12,30-23
2,Абай Аяна,8ENFL,10--8,14--10,NaN,25-18
3,Абай Аяна,8KZFL,7--5,7--6,NaN,30-23
4,Абай Аяна,8KZSL,8--6,8--6,NaN,30-21
...,...,...,...,...,...,...
531,Талғат Алдияр,4RUFL,14--8,9--6,NaN,18-7-
532,Темірбек Айсұлу,4KZFL,10--10,10--7,NaN,15-15
533,Темірбек Айсұлу,4MATH,10--9,10--8,10--8,18-15-
534,Темірбек Айсұлу,4SCIE,10--8,10--9,NaN,15-14-


In [8]:
for key, df in df_dict.items():
    # Testing
    must_have_cols = ['fio', 'subject', 'soch']
    for mhcol in must_have_cols:
        if not 'soch' in df.columns:
            print(f"! Warning ! : No '{mhcol}' column in '{key}'")
            break
    # Testing
    index_list = []
    for index, row in df.iterrows():
        if (pd.isna(df.loc[index, 'subject']) and pd.isna(df.loc[index, 'soch'])): # cheking for empty subject and soch values
            index_list.append(index)
    if len(index_list) > 0:
        # removing rows having with empty subject and soch values simultaneously
        df = df.drop(index=index_list).reset_index() # resetting index order back with step=1
            
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x) # Removing whitespaces
    df['fio'] = df['fio'].map(lambda x: " ".join(str(x).split()[:2]) if len(str(x).split()) > 2 else x) # Leaving only First anf Last names of each student
    # In some tables there is only one name for all subjects. Hence, we need to multiplicate the name to its corresponding subjects
    df['fio'] = df['fio'].ffill()
    # SWE Approach
    # -----------------------------------------
        # name = ''
        # for index, fio in enumerate(df['fio']):
        #     if isinstance(fio, str):
        #         name = fio
        #     else:
        #         df.loc[index, 'fio'] = name
    # -----------------------------------------
    
    # Sometimes, instead of a standard format (mixture of numbers and abbreviations), subjects are written in full words
    lang_db = db[['fio', 'lang']]
    for subject in df['subject'].unique():
        if (not ('4' in subject or '8' in subject)):
            # display(df[df['subject'].isna()])
            df['subject'] = df['subject'].str.lower()
            df['subject'] = df['subject'].map(subject_map)
            # display(df)
            noLang_names = set()
            merged = pd.merge(df, lang_db, on='fio', how='left')
            merged['fio'] = merged['fio'].ffill()
            # display(merged)
            for index, lang in enumerate(merged['lang']):
                if (lang == 1) and (merged.loc[index, 'subject'] == 'RUFL'):
                    merged.loc[index, 'subject'] = 'RUSL'
                elif (lang == 2) and (merged.loc[index, 'subject'] == 'KZFL'):
                    merged.loc[index, 'subject'] = 'KZSL'
            
            merged = merged.drop_duplicates()

            for fio in merged['fio'].unique():
                table = merged[merged['fio'] == fio]
                table.loc[:, 'lang'] = table['lang'].ffill()
                merged.loc[merged['fio'] == fio, 'lang'] = table['lang'].unique()[0]
            
            naLangFio = merged[merged['lang'].isna()]['fio'].unique()
            noLang_names.update(name for name in naLangFio)
            break
    # df = merged
    # -----------------------------------------
    
    df = df.fillna('0')
    
    for col in ['sor1', 'sor2', 'sor3']:
        if not col in df.columns: 
            df[col] = 0

    df[ss_list] = df[ss_list].map(lambda x: ''.join(x.split()) if isinstance(x, str) else x)

    df.dropna(subset=['sor1', 'sor2', 'sor3', 'soch'], how='all', inplace=True)

    for index, char in enumerate(df['subject']):
        if ('4' in char or '8' in char):
            df.loc[index, 'grade'] = df.loc[index, 'subject'][0]
            df.loc[index, 'subject'] = df.loc[index, 'subject'][1:]
            
    df['grade'] = pd.to_numeric(df['grade'], errors='coerce')

    # # for index, col in enumerate(df[ss_list]):
    # #     if isinstance(df[col], str):
    # #         df.at[index, col] = df.at[index, col].replace('/', ' ')
    # for col in ss_list:
    #     df[col] = df[col].astype(str).apply(lambda x: x.replace('/', ' '))

    df[ss_list] = df[ss_list].map(lambda x: str(x).replace('/', ' ')) # replace('\\', ' ').replace('/', ' ').replace('|', ' ').replace('.', '').replace(',', '')

    print(df['sor1'].unique())
    df = df.map(lambda x: np.nan if str(x).isspace() else np.nan if str(x) == '' else x)
    df.dropna(subset=df.columns, how='all', inplace=True)

    for col in ss_list:
        df[f"{col}_max"] = df[col].apply(lambda x: str(x).split()[1] if (len(str(x).split()) > 1 and str(x) != '0') else np.nan)
        df[col] = df[col].apply(lambda x: str(x).split()[0] if len(str(x).split()) >= 1 else x)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[f"{col}_max"] = pd.to_numeric(df[f"{col}_max"], errors='coerce')
    df = df.fillna(0)
    
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

    name = df.loc[2, 'fio']
    school = db.loc[db['fio'].str.contains(name), 'school_ru'].unique()
    
    # Testing
    if len(school) == 1:
        df['school'] = school[0]
    else:
        print(f"!Attention! : '{key}' has no school name")

    df = df[~((df['sor1'] == 0) & (df['sor2'] == 0) & (df['sor3'] == 0))] # removing rows with no values
    
    reindexed_cols = ['school', 'fio', 'grade', 'subject', 'sor1', 'sor1_max', 'sor2', 'sor2_max', 'sor3', 'sor3_max', 'soch', 'soch_max']
    df = df.reindex(columns=reindexed_cols)

    df_dict[key] = df
    
# Testing
for key, df in df_dict.items():
    df = df.drop(columns='school')
    if df.isna().values.any():
        print(f"Error: '{key}' contains NaN values")
        display(df[df.isna()].head(3))
    for subject in df['subject']:
        if len(subject) < 4:
            print(f"Error: 'subject' in '{key}' < 4 characters")
            display(df[df['subject'] == subject])
            break

['10∕9' '17∕14' '10∕8' '12∕9' '20∕20' '13∕10' '13∕12' '8∕7' '10∕6' '17∕12'
 '10∕7' '12∕7' '20∕15' '13∕7' '13∕11' '8∕6' '17∕10' '12∕4' '20∕12' '8∕4'
 '17∕16' '20∕19' '13∕8' '10∕5' '17∕7' '20∕14' '13∕6' '8∕5' '17∕9' '20∕13'
 '20∕18' '0' '20∕11' '17∕13' '20∕9' '20∕16' '17∕8' '13∕9' '20∕10' '12∕6'
 '11∕3' '17∕15' '12∕11' '11∕6' '8∕8' '17 16' '17∕11' '12∕5' '12∕10' '10∕4'
 '11∕5' '11∕7' '12∕8' '10∕10' '9∕6' '9∕10' '8∕10' '9∕7' '9∕9' '8∕9' '9∕8']
!Attention! : 'df26' has no school name


In [22]:
df_dict['df26']['sor3'].unique()

array(['15∕13', nan, '15∕8', '15∕10', '15∕7', '15∕14', '15∕12', '15∕6',
       '15∕11', '15∕9', '15∕15', '12∕15', '14∕15', '10∕5', '10∕10',
       '10∕9', '10∕6', '10∕7', '10∕8'], dtype=object)

In [19]:
for string in df_dict['df26']['sor2']:
    print(string)
    string2 = string.replace('∕', ' ')
    for char in string2:
        print(char)
    break

15∕13
1
5
 
1
3


In [12]:
df_dict['df26']

,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
185,NaN,Әділбек Әбдірахым,8,BIOL,17.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
